# Predicting Click-Through Rate
## Random Forest / XGBoost

Ryan Koch, Sam Kahr, Julia Kang - AMLI 2019

References: https://towardsdatascience.com/mobile-ads-click-through-rate-ctr-prediction-44fdac40c6ff

## Import Libraries and Data

In [5]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import psutil
import random
import datetime as datetime
import matplotlib.pyplot as plt

In [6]:
# read in the Avazu - criteo labs - csv file
# rand_sample_csv is a randomized subset (1% the size) of the sample_csv which is ~400k instances 

df = pd.read_csv('rand_sample_eng.csv')


FileNotFoundError: File b'rand_sample_eng.csv' does not exist

# Data Exploration

In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
# looks like clicks were normalized to 1. 
# What is 0.17 of a click, likely the probability of a click. 0 no click, 1 click

In [ ]:
df.shape # this sample has 4,184 rows of data with 26 columns

In [ ]:
df.dtypes
# avazu: "all integer features are categorical variables, all IDs, no numerical meaning"

In [ ]:
# what do the columns mean?

 - id: ad identifier
 - click: 0/1 for non-click/click
 - hour: format is YYMMDDHH
 - C1 — anonymized categorical variable
 - banner_pos
 - site_id
 - site_domain
 - site_category
 - app_id
 - app_domain
 - app_category
 - device_id
 - device_ip
 - device_model
 - device_type
 - device_conn_type
 - C14-C21 — anonymized categorical variables

In [ ]:
# what are the range of unique values of each column
for col in df.columns.values:
    print( "{}".format(len(df[col].unique())))

In [ ]:
# lets look at an individual user

df[df.id == 13447361190641805430]

# Questions of the data

In [ ]:
# At which time do people click ads most frequently?
# where are most clicks coming from? 
# what nique correlations from any or all combinations of each column relate to click?
# what correlations between seemingly unrelated columns (banner_pos and device type, etc.) exist?
# how are banner_pos, device_type, device_conn_type encoded?

# Data Preprocessing

In [ ]:
#  unnamed column are columns that are created when a dataframe is converted to a csv. 
# 'Unnamed: 0', 'Unnamed: 0.1' are row indexes which were tansposed into columns.
df_new = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1) 

In [7]:
df_new.shape

NameError: name 'df_new' is not defined

In [ ]:
# how many unique values are in each col?

for col in df_new.columns.values:
    print(str(col)+ ": " +  "{}".format(len(df_new[col].unique())) )
    

In [ ]:
# check for missing values

df_new.isnull().sum()


In [ ]:
# summed list of each column for df_new, looking for inconsistencies

for col in df_new.columns.values:
    total = len(df_new[col].unique())
    print(str(col) + " " + "total: " + str(total))

In [ ]:
# tried writing an algo to check for erroneous, inconsistent spelling or abbreviations, formatting issues (e.g., odd/unexpected characters or punctuation)
# couldn't work one out... moving on

Each instance of a column are hashed values of an original ID.
Hashing was done to anonymize the services contributing ad data to this dataset.
For illustrative/descriptive purposes we will treat each hashed value as names or in other fictional contexts (e.g.,'7801e8d9' = 'www.overstock.com'). (Thank you Naomi!)

# Feature Engineering
## Hour & Date 

In [ ]:
# check hour column data type
df_new.hour.dtype

In [ ]:
# separate the date and time
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
df_new['new_hour'] = df_new.hour.astype(str).apply(parse_date)
df_new['new_hour']

In [ ]:
# check if column 'new_hour' was created and parsed to string
df_new.head(3)

In [ ]:
#confirm dtype of new_hour
df_new.new_hour.dtype

In [ ]:
# create new_date & new_time columns from parsed new_hour column
df_new['date'] = [d.date() for d in df_new['new_hour']]
df_new['time'] = [d.time() for d in df_new['new_hour']]

In [ ]:
#check if columns were established properly
df_new.head(3)

In [ ]:
df_new.dtypes

In [ ]:
# drop redundant cols
df_tmp = df_new.drop(['new_hour', 'hour'], axis=1)

In [ ]:
df_tmp.head(2)

In [ ]:
df_tmp.head(2)

In [ ]:
# sannity check of summed columns for unique vals

for col in df_tmp.columns.values:
    total = len(df_tmp[col].unique())
    print(str(col) + " " + "total: " + str(total))

In [ ]:
# save to csv to back up work
df_new.to_csv("rf_date_time_sample.csv")

# Features

 - Target feature : click
 - Site features : site_id, site_domain, site_category
 - App feature: app_id, app_domain, app_category
 - Device feature: device_id, device_ip, device_model, device_type, device_conn_type
 - Anonymized categorical features: C14-C21

# Clicks

In [ ]:
#see what percentage of people click and what percentage of people do not click
# click = 17%, no click = 83%
df_tmp['click'].value_counts()/4184

In [ ]:
#Find the start and end date of the dataset
print(df_tmp['date'].min(), df_tmp['date'].max())

Our dataset ranges from 10/21/2014 to 10/30/2014. It spans a total of 10 days.

# Data Visualization

### Clicks
0 = no click, 1 = click

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='click',data=df_tmp, palette='hls')
plt.show();

In [ ]:
# make groups of summed clicks by date
df_tmp.groupby(['date']).agg({'click': 'sum'}).plot(figsize=(12,6))
plt.ylabel('Number of Clicks')
plt.title('Clicks Trends by Date')


High click rate on Oct. 22nd and OCt. 28
Low click rate on Oct. 24 with lowest rate on Oct. 29

In [ ]:
df_tmp.groupby('time').agg({'click':'sum'}).plot(figsize=(12,6))
plt.ylabel('Number of clicks')
plt.title('Click Trends by Hour of Day');

Highest number of clicks is around 2pm with the lowest number of clicks around 10pm

### Click Impressions

In [ ]:
df_tmp.groupby(['time', 'click']).size().unstack().plot(kind='bar', title="Hour of Day", figsize=(12,6))
plt.ylabel('count')
plt.title('Hourly Impressions vs. Clicks');

## Click Through Rate (CTR)

CTR is the ratio of ad clicks to impressions and is a measure of the rate of clicks on each ad.

### Hourly CTR

In [ ]:
import seaborn as sns

# make a df for click and df for hour
df_click = df_tmp[df_tmp['click'] == 1]
df_hour = df_tmp[['time','click']].groupby(['time']).count().reset_index()
# reformat time to just hour -- this makes the y axis of the viz more readable
df_hour['time'] = df_hour.time.apply(lambda x: x.hour)

In [ ]:
df_click.head(3)

In [ ]:
df_hour.head(3)

In [ ]:
# rename column click to impressions
df_hour = df_hour.rename(columns={'click': 'impressions'})

In [ ]:
df_hour.impressions

In [ ]:
df_hour['clicks'] = df_click[['time','click']].groupby(['time']).count().reset_index()['click']

In [ ]:
df_hour.clicks

In [ ]:
# calculate CTR
# rounding impression to nearest int to make labeling of y axis more readable
df_hour['CTR'] = df_hour['clicks']/df_hour['impressions']*100

In [ ]:
# check CTR hour
df_hour['CTR'][1:3]

In [ ]:
# plot CTR
plt.figure(figsize=(12,6))
sns.barplot(y='CTR', x='time', data=df_hour)
plt.title('Hourly CTR');

Highest CTRs are after 10pm which is odd because after 10pm is the least number of impressions and clicks

## Day of Week (DoW)
### DoW Feature Engineering and Visualizations

In [8]:
# copy df_tmp to new df called df_dow
df_dow = df_tmp

NameError: name 'df_tmp' is not defined

In [ ]:
df_dow.head(3)

In [ ]:
# change date to numeric to make 'dt' method usable
df_dow['date'] = pd.to_datetime(df_dow['date'])

In [ ]:
# make 'day_of_week' date into string representation of day of the week
df_dow['day_of_week'] = df_dow['date'].dt.weekday_name

In [ ]:
df_dow.head(3)

In [ ]:
# check if date vals were encoded properly
df_dow.day_of_week.nunique()

In [ ]:
# remember dataset ranges from 10/21/2014 to 10/30/2014
# 10/21/2014 is a tuesday and is the first day of the dataset
df_dow.day_of_week[0]

In [ ]:
# groupby and reindex based on day of week

In [ ]:
#cats = {7:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_dow.groupby('day_of_week').agg({'click':'sum'}).reindex(cats).plot(figsize=(12,6))
# change ticks to match x axis label
ticks = list(range(0, 7, 1)) # points on the x axis where you want the label to appear
labels = "Mon Tues Weds Thurs Fri Sat Sun".split()
plt.xticks(ticks, labels)
plt.title('Click Trends by Day of Week');

Looks like highest number of clicks are on Tuesday with the lowest number of clicks on Friday

In [ ]:
# group by day_of_week and click and plot bar chart of clicks/impressions by day_of_week
df_dow.groupby(['day_of_week','click']).size().unstack().reindex(cats).plot(kind='bar', title="Day of the Week", figsize=(12,6))

# change ticks to match x axis label
ticks = list(range(0, 7, 1))
labels = "Mon Tues Weds Thurs Fri Sat Sun".split()

# plot
plt.xticks(ticks, labels)
plt.title('Impressions vs. Clicks by Day of Week');

Tuesday - Thurs. have most number of impressions and clicks. Mon. & Friday have the least amount of clicks & impressions.

## Day of Week CTR

In [ ]:
# make df of positive hits / clicks = 1
df_click = df_dow[df_dow['click'] == 1]

In [ ]:
# make df_dayofweek with just dow and click data
df_dayofweek = df_dow[['day_of_week','click']].groupby(['day_of_week']).count().reset_index()

In [ ]:
df_dayofweek.head(3)

In [ ]:
# rename clicks to impressions to disambiguate 'click'. Impressions are clicks = 0 or false postivies
df_dayofweek = df_dayofweek.rename(columns={'click': 'impressions'})

In [ ]:
df_dayofweek.head(3)

In [ ]:
# add 'clicks' column of only positive hits for click (clicks = 1)
df_dayofweek['clicks'] = df_click[['day_of_week','click']].groupby(['day_of_week']).count().reset_index()['click']

In [ ]:
df_dayofweek['clicks']

In [ ]:
df_dayofweek.head(3)

In [ ]:
# calculate CTR 
df_dayofweek['CTR'] = df_dayofweek['clicks']/df_dayofweek['impressions']*100

In [ ]:
df_dayofweek['CTR']

In [ ]:
# plot day of week CTR
plt.figure(figsize=(12,6))
sns.barplot(y='CTR', x='day_of_week', data=df_dayofweek, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Day of week CTR');

Tues & Weds. have the highest number of impressions and clicks but the lowest CTR. Saturday & Monday have the highest CTR.

## C1 

In [ ]:
# check out distribution of C1 across all columns
print(df_tmp.C1.value_counts()/len(df_tmp))

C1 (value = 1005) is about 93% of all the data we are using. 

### C1 Data Exploration and Feature Engineering
Investigate value of C1 in relation to CTR.

In [ ]:
# check unique vals for C1
C1_values = df_tmp.C1.unique()

In [ ]:
print(C1_values) # there are 5 unique vals

In [ ]:
# sort C1 vals
C1_values.sort()

In [ ]:
# instantiate empty list for avg CTR vals
ctr_avg_list=[]

In [ ]:
# look up C1s click mean vals, append values to ctr_avg list, print results
for i in C1_values:
    ctr_avg=df_tmp.loc[np.where((df_tmp.C1 == i))].click.mean()
    ctr_avg_list.append(ctr_avg)
    print("for C1 value: {},  click through rate: {}".format(i,ctr_avg))

In [ ]:
# group C1 with click and plot distribution
df_tmp.groupby(['C1', 'click']).size().unstack().plot(kind='bar', figsize=(12,6), title='C1 distribution');

It appears that C1 values of '1005' have the most clicks and impressions

### C1 Feature engineering for group C1 with click and plot distribution

In [ ]:
# group C1 & click
df_c1 = df_tmp[['C1','click']].groupby(['C1']).count().reset_index()

In [ ]:
# disambiguate click from impressions
df_c1 = df_c1.rename(columns={'click': 'impressions'})

In [ ]:
# add clicks as true positives (hits)
df_c1['clicks'] = df_click[['C1','click']].groupby(['C1']).count().reset_index()['click']

In [ ]:
df_c1.head(3)

In [ ]:
df_click.head(3)

In [ ]:
# calculate CTR
df_c1['CTR'] = df_c1['clicks']/df_c1['impressions']*100

In [ ]:
df_c1['CTR'] 

In [ ]:
# confirm cols were created properly with CTR 
df_c1.head(2)

In [ ]:
# plot CTR by C1

plt.figure(figsize=(12,6))
sns.barplot(y='CTR', x='C1', data=df_c1)
plt.title('CTR by C1');

### C1 values and CTR pairs:

C1=1005: 93% of the data and 0.17 CTR

C1=1002: ~1.9% of the data and 0.21 CTR

C1=1010: 1% of the data and 0.095 CTR

C1=1012: 2.6% of data and no CTR


C1 = 1002 has a much higher than average CTR, and C1=1012 has a much lower than average CTR, it seems these two C1 values are important for predicting CTR.

# Random Forest

In [ ]:
rf_features = df_tmp.drop('click', axis = 1)

In [ ]:
rf_features.head(3)

In [ ]:
# RF is supervised because we have both the features (data as df_tmp) and the targets (clicks)
# We give the random forest both the features and targets and it must learn how to map the data to a prediction



# Feature Engineering

In [ ]:
df_tmp.head(3)

# 1. Know your data

Look at Summary statistics and visualizations
Percentiles can help identify the range for most of the data
Averages and medians can describe central tendency
Correlations can indicate strong relationships

# 2. Visualize the data

Box plots can identify outliers
Density plots and histograms show the spread of data
Scatter plots can describe bivariate relationships
Clean your data

Deal with missing value. Missing data affects some models more than others. Even for models that handle missing data, they can be sensitive to it (missing data for certain variables can result in poor predictions)
Choose what to do with outliers
Outliers can be very common in multidimensional data.
Some models are less sensitive to outliers than others. Usually tree models are less sensitive to the presence of outliers. However regression models, or any model that tries to use equations, could definitely be effected by outliers.
Outliers can be the result of bad data collection, or they can be legitimate extreme values.

# 3. Does the data need to be aggregated

Augment your data

Feature engineering is the process of going from raw data to data that is ready for modeling. It can serve multiple purposes:
Make the models easier to interpret (e.g. binning)
Capture more complex relationships (e.g. NNs)
Reduce data redundancy and dimensionality (e.g. PCA)
Rescale variables (e.g. standardizing or normalizing)

### 4. Different models may have different feature engineering requirements. Some have built in feature engineering.



# 1. Know your data¶
Look at Summary statistics and visualizations Percentiles can help identify the range for most of the data Averages and medians can describe central tendency Correlations can indicate strong relationships

# 2. Visualize the data¶
Box plots can identify outliers Density plots and histograms show the spread of data Scatter plots can describe bivariate relationships Clean your data

Deal with missing value. Missing data affects some models more than others. Even for models that handle missing data, they can be sensitive to it (missing data for certain variables can result in poor predictions) Choose what to do with outliers Outliers can be very common in multidimensional data. Some models are less sensitive to outliers than others. Usually tree models are less sensitive to the presence of outliers. However regression models, or any model that tries to use equations, could definitely be effected by outliers. Outliers can be the result of bad data collection, or they can be legitimate extreme values.

# 3. Does the data need to be aggregated
Augment your data

Feature engineering is the process of going from raw data to data that is ready for modeling. It can serve multiple purposes: Make the models easier to interpret (e.g. binning) Capture more complex relationships (e.g. NNs) Reduce data redundancy and dimensionality (e.g. PCA) Rescale variables (e.g. standardizing or normalizing)

In [4]:
df_tmp.head(3)

NameError: name 'df_tmp' is not defined